In [1]:
import sys

from morpheus.project.types.geometry.MultiPolygon import MultiPolygon

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


In [2]:
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid
from morpheus.project.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96609040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(20)
length_unit = LengthUnit.meters()

grid = Grid.cartesian_from_polygon(
  polygon=polygon,
  n_rows=1000,
  n_cols=1000,
  rotation=rotation,
)
polygon

Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96609040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon')

## Performance of ActiveCells Calculation with a LineString 

In [3]:
from morpheus.project.types.geometry import LineString

linestring_within = LineString(
  type='LineString',
  coordinates=[
    (13.923, 50.965),
    (13.924, 50.965),
  ]
)

In [4]:
%%time
from morpheus.project.types.discretization.spatial import ActiveCells

ac = ActiveCells.from_linestring(linestring=linestring_within, grid=grid)
len(ac)

CPU times: user 244 ms, sys: 3.62 ms, total: 247 ms
Wall time: 188 ms


295

## Performance of ActiveCells Calculation with a Polygon

In [5]:
from morpheus.project.types.geometry import Polygon

polygon_within = Polygon(
  type='Polygon',
  coordinates=[[
    (13.923, 50.965),
    (13.924, 50.9655),
    (13.925, 50.965),
    (13.925, 50.966),

    (13.923, 50.966),
    (13.923, 50.965),
  ]]
)

In [6]:
import folium

m = folium.Map(location=[50.9655, 13.9235], zoom_start=15)
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(polygon_within.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m


In [7]:
%%time
active_cells = ActiveCells.from_polygon(polygon=polygon_within, grid=grid)
len(active_cells)

CPU times: user 622 ms, sys: 11.1 ms, total: 634 ms
Wall time: 677 ms


318046

## Performance of ActiveCells Calculation with a MultiPolygon

In [8]:
multi_polygon_within = MultiPolygon(
  type='MultiPolygon',
  coordinates=[
    [[
      (13.923, 50.965),
      (13.924, 50.9655),
      (13.925, 50.965),
      (13.925, 50.966),
      (13.923, 50.966),
      (13.923, 50.965),
    ]],
    [[
      (13.9235, 50.9655),
      (13.924, 50.966),
      (13.9245, 50.9655),
      (13.9235, 50.9655),
    ]]
  ]
)

active_cells = ActiveCells.from_multipolygon(multipolygon=multi_polygon_within, grid=grid)